# A script to adjust data

## Import libraries

In [1]:
from PIL import Image
import requests
from io import BytesIO
import csv

# Verification of images

A dataset that I use comes from 2017, since it provides links to resource on the internet, some of the images are no longer available. The following function checks if an image pointed by argument "url" can be loaded and if so returns its width and height.

In [69]:
from PIL import UnidentifiedImageError

def verify_image(url):
    if url == '':
       return (False, 0, 0) 

    try:
        response = requests.get(url)
        img = Image.open(BytesIO(response.content))
        (w, h) = img.size
        return (True, w, h)
    except UnidentifiedImageError as error: # Throws exception if an image cannot be fetched
        return (False, 0, 0)

print (verify_image('https://c3.staticflickr.com/3/2820/11394761256_a8560c6701_z.jpg'))
print (verify_image('https://c1.staticflickr.com/9/8087/8563450894_0b57eb01b3_z.jpg'))

(False, 0, 0)
(True, 640, 424)


## Generate .sql scripts
We are about to run scripts that are going to generate .sql scripts.

### Target file name
 Let's specify names of the target files first.

In [65]:
filename_images = "sampledata_images.sql"
filename_belong = "sampledata_belong.sql"
filename_areas = "sampledata_areas.sql"
filename_class = "sampledata_class.sql"
filename_labels = "sampledata_labels.sql"


### Image & DatasetBelonging
A first chunk generates script to fill both those tables

In [79]:
# IMAGES
LINES = 100

# clear contents of both files
open(filename_images, 'w').close()
open(filename_belong, 'w').close()

# open target files
f = open(filename_images, "a")
f2 = open(filename_belong, "a")


# source file
file1 = open('data/2017_11/validation/images.csv', 'r', encoding="utf-8")
line = file1.readline() 

count = 0
id_dict = {}

while True:
    line = file1.readline() 
  
    if not line: 
        break
        
    sep = line.strip().split(",")
    
    img_id = sep[0]
    url = sep[-1]
    (pred, w, h) = verify_image(url)
    
    if pred:
        count += 1
        id_dict[img_id] = [url, w, h, -count]
        print("Line {}: {}, {}".format(count, sep[0], sep[-1]))
        f.write("INSERT INTO Image VALUES ('{}', '{}');\n".format(-count, url))
        f2.write("INSERT INTO DatasetBelonging VALUES ('{}', '{}');\n".format(-count, (-count%3) + 1))
        
        

    if count >= LINES:
        break 
        

file1.close() 
f.close()
f2.close()

Line 1: 0001eeaf4aed83f9, https://c4.staticflickr.com/6/5606/15611395595_0594841ba5_z.jpg
Line 2: 0004886b7d043cfd, https://c3.staticflickr.com/3/2808/10351094034_e7c52e0402_z.jpg
Line 3: 000595fe6fee6369, https://c6.staticflickr.com/9/8089/8416776003_b45b381eae_z.jpg
Line 4: 00075905539074f2, https://c7.staticflickr.com/1/568/21452126474_85567a98ee_z.jpg
Line 5: 0007cebe1b2ba653, https://c5.staticflickr.com/2/1244/677743874_168dcb5ed1_z.jpg
Line 6: 0007d6cf88afaa4a, https://c7.staticflickr.com/4/3750/9031823026_3e55960d76_z.jpg
Line 7: 0008e425fb49a2bf, https://c5.staticflickr.com/9/8078/8343384446_ebe3bc86ce_z.jpg
Line 8: 0009bad4d8539bb4, https://c7.staticflickr.com/6/5551/14057086448_2c780a869b_z.jpg
Line 9: 000a1249af2bc5f0, https://c1.staticflickr.com/9/8087/8563450894_0b57eb01b3_z.jpg
Line 10: 000ada55d36b4bcb, https://c1.staticflickr.com/6/5019/5442850400_2c96b8caac_z.jpg
Line 11: 000c4d66ce89aa69, https://c3.staticflickr.com/4/3113/3197300261_4e5e9c5e3e_z.jpg
Line 12: 00101a01

Line 92: 0098263ae56016d3, https://c3.staticflickr.com/5/4085/5047853382_b2c736269e_z.jpg
Line 93: 0098755e846b745b, https://c3.staticflickr.com/1/724/21492181710_dabd9b7444_z.jpg
Line 94: 00991ee13e849b07, https://c1.staticflickr.com/1/15/19283436_772222a19e_z.jpg
Line 95: 009be28128a2bb65, https://c6.staticflickr.com/9/8433/7728243022_84838b08f4_z.jpg
Line 96: 009dfe7e81b732cb, https://c1.staticflickr.com/3/2657/4034890388_fa7f08fbaa_z.jpg
Line 97: 00a0b916fd5941a3, https://c4.staticflickr.com/8/7293/8745465414_14f65f46d2_z.jpg
Line 98: 00a159a661a2f5aa, https://c6.staticflickr.com/8/7093/13645764903_0e2239e2a6_z.jpg
Line 99: 00a1f2a6e7f78ac5, https://c4.staticflickr.com/6/5292/5516505342_43268015f4_z.jpg
Line 100: 00a300e8b0cef4d3, https://c3.staticflickr.com/4/3934/15260830600_feb38b2d9e_z.jpg


### ImageArea & Classification
Second chunk is a bit more complicated. It selects only those boxes, which are drawn on selected images. It also keeps tracks of labels.

In [76]:
# BOXES & CLASSIFICATIONS
LIMIT = 5000

# clear contents of both files
open(filename_areas, 'w').close()
open(filename_class, 'w').close()

# open target files
f = open(filename_areas, "a")
f_class = open(filename_class, "a")

# open input file
file2 = open('data/2017_11/validation/annotations-human-bbox.csv', 'r')
line = file2.readline() 
print (line)

# set up counters
count2 = 0
how_many = 0
different_labels = 1


label_dict = {}

while True:
    line = file2.readline() 
    count2 += 1
    
    if (not line) or (count2 > LIMIT): 
        break

    sep = line.strip().split(",")

    label_id = sep[2]
    img_id = sep[0]
    area = [float(sep[i]) for i in range(4,8)]
    
    if img_id not in id_dict:
        continue
    
    
    val = id_dict[img_id]

    how_many += 1

    x = round(area[0]*val[1])
    y = round(area[2]*val[2])
    width = round((area[1] - area[0])*val[1])
    height = round((area[3] - area[2])*val[2])
    area_id = -how_many
    image_id = val[3]

    print (label_id, 
           img_id, 
           round(area[0]*val[1]),
           round(area[2]*val[2]), 
           round((area[1] - area[0])*val[1]), 
           round((area[3] - area[2])*val[2]),
           val[0]
          )
    f.write("insert into ImageArea values ({}, {}, {}, {}, {}, {}, 0);\n".format(area_id, x, y, width, height, image_id))

    # handle classification
    lid = -different_labels
    if label_id in label_dict:
        lid = label_dict[label_id]
    else:
        label_dict[label_id] = lid
        different_labels += 1

    f_class.write("insert into Classification values ({}, {}, 1, {});\n".format(area_id, area_id, lid))

file2.close()
f.close()
f_class.close()
print(how_many)


ImageID,Source,LabelName,Confidence,XMin,XMax,YMin,YMax,IsOccluded,IsTruncated,IsGroupOf,IsDepiction,IsInside

/m/0cmf2 0001eeaf4aed83f9 14 20 602 204 https://c4.staticflickr.com/6/5606/15611395595_0594841ba5_z.jpg
/m/02wbm 000595fe6fee6369 0 0 640 426 https://c6.staticflickr.com/9/8089/8416776003_b45b381eae_z.jpg
/m/02xwb 000595fe6fee6369 90 288 25 24 https://c6.staticflickr.com/9/8089/8416776003_b45b381eae_z.jpg
/m/02xwb 000595fe6fee6369 137 127 25 24 https://c6.staticflickr.com/9/8089/8416776003_b45b381eae_z.jpg
/m/02xwb 000595fe6fee6369 149 208 36 24 https://c6.staticflickr.com/9/8089/8416776003_b45b381eae_z.jpg
/m/02xwb 000595fe6fee6369 157 282 29 23 https://c6.staticflickr.com/9/8089/8416776003_b45b381eae_z.jpg
/m/02xwb 000595fe6fee6369 157 233 29 24 https://c6.staticflickr.com/9/8089/8416776003_b45b381eae_z.jpg
/m/02xwb 000595fe6fee6369 158 162 23 25 https://c6.staticflickr.com/9/8089/8416776003_b45b381eae_z.jpg
/m/02xwb 000595fe6fee6369 158 259 30 28 https://c6.staticflickr.com

/m/01jfm_ 008637722500f239 152 245 68 40 https://c7.staticflickr.com/6/5289/5340010485_edeaa88c83_z.jpg
/m/01jfm_ 008637722500f239 510 157 36 19 https://c7.staticflickr.com/6/5289/5340010485_edeaa88c83_z.jpg
/m/083wq 008637722500f239 80 324 77 43 https://c7.staticflickr.com/6/5289/5340010485_edeaa88c83_z.jpg
/m/083wq 008637722500f239 253 180 25 19 https://c7.staticflickr.com/6/5289/5340010485_edeaa88c83_z.jpg
/m/083wq 008637722500f239 352 258 67 126 https://c7.staticflickr.com/6/5289/5340010485_edeaa88c83_z.jpg
/m/083wq 008637722500f239 359 263 57 113 https://c7.staticflickr.com/6/5289/5340010485_edeaa88c83_z.jpg
/m/083wq 008637722500f239 458 234 52 94 https://c7.staticflickr.com/6/5289/5340010485_edeaa88c83_z.jpg
/m/083wq 008637722500f239 518 226 20 6 https://c7.staticflickr.com/6/5289/5340010485_edeaa88c83_z.jpg
/m/083wq 008637722500f239 606 188 33 66 https://c7.staticflickr.com/6/5289/5340010485_edeaa88c83_z.jpg
/m/083wq 008637722500f239 609 191 30 57 https://c7.staticflickr.com/6/5

### Labels
Now the final script.

In [77]:
# LABELS

# clear contents of file
open(filename_labels, 'w').close()

# open target file
f = open(filename_labels, "a")

# open input file
file = open('data/2017_11/class-descriptions.csv', 'r', encoding="utf-8")
readCSV = csv.reader(file, delimiter=',')

# set up counters
count = 0


for row in readCSV:
    count += 1

    [label_id, name] = [row[0], row[1]]
    
    if label_id in label_dict:
        print(label_id, name)
        #print(label_dict[label_id])
        f.write("insert into Label values ({}, '{}', null, 0);\n".format(
            label_dict[label_id],
            name
        ))

    
    

file.close()
f.close()


/m/0138tl Toy
/m/014sv8 Eye
/m/015x4r Cucumber
/m/018xm Ball
/m/0199g Bike
/m/019jd Boat
/m/019w40 Surfboard
/m/01_bhs Fast food
/m/01bl7v Boy
/m/01bqk0 Bicycle wheel
/m/01f8m5 Blue jay
/m/01g317 Person
/m/01jfm_ Licence plate
/m/01lrl Carnivore
/m/01n4qj Shirt
/m/01nkt Cheese
/m/01prls Land vehicle
/m/01s55n Suitcase
/m/01xs3r Jet ski
/m/01z1kdw Juice
/m/024g6 Cocktail
/m/02522 Computer monitor
/m/025dyy Box
/m/0271t Drink
/m/0283dt1 Mouth
/m/02dgv Door
/m/02jnhm Tin can
/m/02p0tk3 Human body
/m/02p5f1q Coffee cup
/m/02pkr5 Plumbing fixture
/m/02vqfm Coffee
/m/02wbm Food
/m/02xwb Fruit
/m/02y6n French fries
/m/035r7c Leg
/m/03bt1vf Woman
/m/03q69 Hair
/m/03vt0 Insect
/m/04bcr3 Table
/m/04brg2 Tableware
/m/04hgtk Head
/m/04m9y Lizard
/m/04rky Mammal
/m/04_sv Motorcycle
/m/04yx4 Man
/m/052lwg6 Baked goods
/m/05r655 Girl
/m/05s2s Plant
/m/05y5lj Sports equipment
/m/0663v Pizza
/m/071qp Squirrel
/m/07c52 Television
/m/07j7r Tree
/m/07mhn Trousers
/m/07yv9 Vehicle
/m/080hkjn Handbag
/m/083

## Concatenate files
We will create one file from all those that we created plus the hand-written part. An order is important.

In [80]:
output_filename = "combined_data.sql"

filenames = [
    "sample_data.sql",
    filename_images,
    filename_belong,
    filename_areas,
    filename_labels,
    filename_class
]

with open(output_filename, 'w') as outfile:
    for fname in filenames:
        with open(fname) as infile:
            for line in infile:
                outfile.write(line)